In [22]:
import os
import importlib
import logging
import time
importlib.reload(logging)
import framework
importlib.reload(framework)
import bert_qa
importlib.reload(bert_qa)
import infer_bert_qa
importlib.reload(infer_bert_qa)
import bert_utils
importlib.reload(bert_utils)
import pandas as pd
from framework import DataCuration, FeatureEngineering
from bert_qa import TaskQA, FeatureEngineeringQA, BERTQA

# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)
ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Set up the task details. This notebook handles Question Answering for CARTA dataset.

Example

context = "New Zealand (Māori: Aotearoa) is a sovereign island country in the southwestern Pacific Ocean. It has a total land area of 268,000 square kilometres (103,500 sq mi), and a population of 4.9 million. New Zealand's capital city is Wellington, and its most populous city is Auckland."

questions = "How many people live in New Zealand?", "What's the largest city?"


In [23]:
DATASET = 'carta' # supports w2 and resume
TASK_CONFIG = {
    'task': 'qa'
}

task = TaskQA(TASK_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [24]:
CARTA_DATA = [
   '/Users/ahsaasbajaj/Documents/Data/QA_model/data'
]
CARTA_GOLDEN = [
   '/Users/ahsaasbajaj/Documents/Data/CARTA/Annotated Samples/golden/output.csv'
]

GOLDEN_CONFIG = {
    'path': CARTA_GOLDEN,
    'is_local': True,
    'index_field_name':'filename',
    'file_type': 'csv',
    'identifier': 'file'
}
DATASET_CONFIG = {
    'path': CARTA_DATA,
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': True
}

data = DataCuration(ACCESS_TOKEN, DATASET_CONFIG, GOLDEN_CONFIG)

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/QA_model/data
INFO:root:1 files loaded
INFO:root:Loading goldens from /Users/ahsaasbajaj/Documents/Data/CARTA/Annotated Samples/golden/output.csv
INFO:root:Total files Goldens: (4, 9)
INFO:root:Total files found in the source with unique index: (1, 9)
INFO:root:Processing 1 IBOCR files to txt


In [25]:
data.dataset

{'annotated_AOI_4.pdf': <instabase.ocr.client.libs.ibocr.ParsedIBOCR at 0x14b6cb4a8>}

In [26]:
data.golden

,Number of authorized shares / share class,Number of authorized shares / preferred share type,Cumulative dividends,Dividend rate,Original Issue Price,Liquidation preference / preferred share type,Seniority (Preferred share class),Participation (Preferred share class),Conversion price (Preferred share class)
filename,,,,,,,,,
annotated_AOI_4.pdf,"Common Stock: 16,000,000 shares; Preferred Sto...","Series Seed Preferred Stock: 1,820,119 shares;...",False,Series A Preferred Stock: $0.1044 per annum pe...,Series Seed Preferred Stock: $0.795 per share;...,Series Seed Preferred Stock: $0.795 per share;...,Series Seed Preferred Stock: 1; Series A Prefe...,Series A Preferred Stock: $2.6098 per share,Series Seed Preferred Stock: $0.795 per share;...


In [27]:
# open_queries = [ 
#                 "Who is incorporating the company?",
#                 "How many shares are being created?",
#                 "What are the number of authorized shares?",
#                 "What are the Preferred stocks?",
#                 "What are the Non-cumulative dividends?",
#                 "What are the Common stocks?",
#                 "What is the Dividend rate per annum per preferred share type?",
#                 "What is the original issue price per share?",
#                 "What is the seniority of preferred share?",
#                 "What is the liquidation preference?",
#                 "What is the conversion price"
#                 ]

In [45]:
query = "What are the Preferred stocks?"

NUM_FILES = len(data.dataset.keys())
stime = time.time()

DATA_ARGS = {
    'task': task,
    'dataset': data
}

TRAINING_ARGS = {
'model_file_or_path': "/Users/ahsaasbajaj/Documents/Data/QA_model/model", # finetuned checkpoint available directly
'gpu': False,
'output_dir': '../outputs/bert_qa'
}

model = BERTQA(DATA_ARGS, TRAINING_ARGS)
output = model.predict(query)

etime = time.time()
logging.info('Total time {} seconds'.format(etime - stime))

INFO:root: Total number of Files: 1
INFO:root:File name: annotated_AOI_4.pdf
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2221.56it/s]
INFO:root:Total time 168.7658941745758 seconds
What are the Preferred stocks? : Series A Preferred Stock and Series A-1 Preferred Stock


In [64]:
output = output.set_index('filename')
filenames = output.index.to_list()

for filename in filenames:
    print("filename: ", filename)

    for col in output1.columns.to_list():
        print("query: ", col)
        answer = output.loc[filename, col]
        print("answer: ", answer)
    

filename:  annotated_AOI_4.pdf
query:  What are the Preferred stocks?
answer:  Series A Preferred Stock and Series A-1 Preferred Stock
